In [11]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import illustris_python as il
import pandas as pd

#intitial setup
basePath = "../data/tng-100-3/output"
fields = {"stars": ["Coordinates", "Potential", "Masses", "Velocities"],
        "gas": ["Coordinates", "Masses", "Velocities"],
        "dm": ["Coordinates", "Potential"]
        }
indices = [1893, 2164, 3043, 3229, 3403, 3605, 3760] #np.genfromtxt("./data/tng-100-3/cutdata/early_type_indices.csv")
DM_PARTICLE_MASS = 0.03235675 #found in header of snapshot
N = len(indices)
stars = [0]*N
gas = [0]*N
dm = [0]*N
particle_lists= [stars, gas, dm]
group_cat = pd.DataFrame({"id": indices})


In [5]:
def radius(df):
    x = np.array(df["x"])
    y = np.array(df["y"])
    z = np.array(df["z"])
    r = (x**2 + y**2 + z**2)**(1/2)
    return r

In [12]:
#Load all particles
for i in range(N):
    stars[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'stars', fields["stars"]))
    gas[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'gas', fields["gas"]))
    dm[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'dm', fields["dm"]))
    dm[i]["Masses"] = [DM_PARTICLE_MASS]*len(dm[i]["Potential"])


subhaloFields = ["SubhaloMass", 'SubhaloMassType', "SubhaloMassInHalfRadType", "SubhaloHalfmassRadType"]
subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhaloFields)
df_subhalos = il.pandasformat.dict_to_pandas(subhalos)

OSError: Unable to open file (unable to open file: name = '../data/tng-100-3/output/../postprocessing/offsets/offsets_099.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [8]:
#calculate radius

for i in range(N):
    #find center of halo
    min_pot_value = stars[i]["Potential"].min()
    min_pot_pos = stars[i][stars[i]["Potential"] == min_pot_value]["Coordinates"].values[0]
    for particle in particle_lists:
        #Calculate new coordinates
        particle[i]["x"] = particle[i]["X"] - min_pot_pos[0]
        particle[i]["y"] = particle[i]["Y"] - min_pot_pos[1]
        particle[i]["z"] = particle[i]["Z"] - min_pot_pos[2]
        particle[i]["r"] = radius(particle[i])


In [9]:
#Calculate total mass for each particle type
stellar_mass = np.zeros(N)
gas_mass = np.zeros(N)
dm_mass = np.zeros(N)

for i in range(N):
    stellar_mass[i] = stars[i]["Masses"].sum() #or load from snapshot to save time
    gas_mass[i] = gas[i]["Masses"].sum()
    dm_mass[i] = dm[i]["Masses"].sum()

total_mass = stellar_mass + gas_mass + dm_mass
#save value to catalogue
group_cat["SubhaloMassStellar"] = stellar_mass
group_cat["SubhaloMassGas"] = gas_mass
group_cat["SubhaloMassDM"] = dm_mass
group_cat["SubhaloMass"] = total_mass


In [61]:
halfmass_rad = np.zeros(N)
#Calculate half mass radius
for i in range(N):
    temp = stars[i].copy(deep=True)
    temp = temp.sort_values(by="r")
    temp_mass = 0
    n = temp["count"][0]
    for j in range(n):
        if temp_mass < (group_cat["SubhaloMassStellar"][i]/2):
            temp_mass = temp_mass + temp["Masses"][j]
        else:
            m1 = temp["Masses"][j-1]
            m2 = temp["Masses"][j]
            M = m1+m2
            halfmass_rad[i] = (m1*temp["r"][j-1] + m2*temp["r"][j])/M #some uncertainty here, now center of mass
            break  
group_cat["SubhaloHalfmassRadStellar"] = halfmass_rad

In [62]:
print(group_cat)
print("MAss: ", df_subhalos["SubhaloMassStellar"][1893])
print("Radius: ", df_subhalos["SubhaloHalfmassRadStellar"][2164])
#print(df_subhalos["SubhaloMassStellar"][1893])

     id  SubhaloMassStellar  SubhaloMassGas  SubhaloMassDM   SubhaloMass  \
0  1893           56.558525     2261.504639   13384.078427  15702.141591   
1  2164           87.635010     2214.491699   13637.172925  15939.299634   
2  3043           61.568253     1921.545654   11098.591747  13081.705654   
3  3229           51.169834     1540.257568    8965.246506  10556.673909   
4  3403           18.035259     1262.919922    7397.464899   8678.420080   
5  3605           16.470587      304.344513    2952.941718   3273.756818   
6  3760           38.428741     1201.637451    7368.214397   8608.280589   

   SubhaloHalfmassRadStellar  
0                  24.917713  
1                  33.964071  
2                  31.957937  
3                  43.473352  
4                  16.029240  
5                  18.897331  
6                  10.059874  
MAss:  56.558517
Radius:  35.24243
